# step04 Randomly select 5 dates from one patient's records
- Randomly select 5 documents from one patient's records
- For each department, a bundle of documents for each patient is prepared in date order
- result example
<pre>
file_loc	querydate	option1date	option2date	option3date	option4date
./data/03_pool/신장내과/00056184.txt	2011 - 02 - 15	0	2012 - 01 - 06	1	2012 - 12 - 18	4	2012 - 06 - 21	3	2012 - 03 - 30	2
</pre>

In [ ]:
import glob, os
import random

target_pts = []
pools = []
train_pool = "../preprocessing/01_data4finetune/pts_SNUH_visit_2011to2020_heldout_train.txt"
test_pool  = "../preprocessing/01_data4finetune/pts_SNUH_visit_2011to2020_heldout_test.txt"

def target_ptnums(target_file):
    target_pool = []
    file = open(target_file, "r")
    lines = file.readlines()
    for l in range(len(lines)):
        line = lines[l].strip("\n")
        ptnum = line.split("/")[-1].replace(".txt", "")
        target_pool.append(ptnum)
    file.close()
    
    return target_pool

train_pool = target_ptnums(train_pool)
test_pool  = target_ptnums(test_pool)
pools = train_pool + test_pool
print("len(pools): ", len(pools))

In [ ]:
def read_documents(path):
    
    file = open(path, "r")
    lines = file.readlines()
    file.close()
    
    date2doc = {}
    date = ""
    doctype = ""
    
    document = {}
    for l in range(len(lines)):
        line = lines[l].strip("\n")
        
        if len(line)==0:
            if date in date2doc:
                continue
            else:
                date2doc[date] = document
                document = {}
                date = ""
                
        else:
            section = line.split("    || ")[0]
            content = line.split("    || ")[1]
        
            if "DATE"==section:
                date = content
            elif "doctype"==section:
                doctype = content

            else:
                key = doctype+"\t"+section
                if key in document:
                    content_tmp = document[key]
                    content_tmp.append(content)
                    document[key] = content_tmp
                else:
                    document[key] = [content]
                
    if len(document)>0:
        if date not in date2doc:
            date2doc[date] = document
            document = {}
            date = ""
    
    return date2doc


def set_dates(train_pool, test_pool):
    departments = glob.glob("./data/03_pool/*")
    departments.sort()
    print("departments: ", departments)
    
    for d in range(len(departments)):
        patient_files = glob.glob(departments[d]+"/*.txt")
        patient_files.sort()
        department = departments[d].split("/")[-1]
        print("department: ", department)
        
        questions_train = ["file_loc\tquery\topt1\tidx1\topt2\tidx2\topt3\tidx3\topt4\tidx4\t"]
        questions_test  = ["file_loc\tquery\topt1\tidx1\topt2\tidx2\topt3\tidx3\topt4\tidx4\t"]
        
        # FILE LOOP
        for p in range(len(patient_files)):
            print("patient_files[p]: ", patient_files[p])
            
            # train / test mode
            patient = patient_files[p].split("/")[-1].replace(".txt", "")
            if patient in train_pool:
                mode = "train"
            elif patient in test_pool:
                mode = "test"
            
            # read documents
            date2doc_p = read_documents(patient_files[p])
            
            # target patient must have five or more records
            if len(date2doc_p)<5:
                continue
            
            # 무작위 문서 번호를 선택
            dates = [key for key in date2doc_p]
            #print("dates: ", dates)
            
            
            if mode=="train":
                n_max_sample = min(len(dates)-5, 10)
                option_comb = 3 
                
            elif mode=="test":
                n_max_sample = min(len(dates)-5, 1)
                option_comb = 1 
            
            for i in range(n_max_sample):
                random.shuffle(dates)
                selected_dates = dates[:5]
                selected_dates.sort()
                
                # shuffle
                for q in range(option_comb):
                    quest = []
                    quest.append(patient_files[p])
                    
                    # query
                    query_date = selected_dates[0]
                    quest.append(query_date)
                    quest.append("0")

                    # options
                    order = [k for k in range(1, 5)]
                    random.shuffle(order)
                    for o in range(len(order)):
                        cand_date = selected_dates[order[o]]
                        quest.append(str(cand_date))
                        quest.append(str(order[o]))
                
                if mode=="train":
                    questions_train.append("\t".join(quest))
                elif mode=="test":
                    questions_test.append("\t".join(quest))
                
        
        outpath = "./data/04_selected_idx/train"
        if not os.path.exists(outpath):
            os.makedirs(outpath)
        file = open(outpath+"/"+str(department)+".txt", "w")
        file.write("\n".join(questions_train))
        file.close()
        
        
        outpath = "./data/04_selected_idx/test"
        if not os.path.exists(outpath):
            os.makedirs(outpath)
        file = open(outpath+"/"+str(department)+".txt", "w")
        file.write("\n".join(questions_test))
        file.close()
        
        print("written "+str(department))
        
    
set_dates(train_pool, test_pool)
